In [1]:
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
features_ventana1 = pd.read_csv('../xgb/features_clicks_gonzalo_ventana1.csv').set_index('ref_hash')
aux_ventana1 = pd.read_csv('../xgb/features_events_gonzalo_ventana1.csv').set_index('ref_hash')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer', lsuffix = '_clicks', rsuffix = '_events')
aux_ventana1 = pd.read_csv('../xgb/features_installs_gonzalo_ventana1.csv').set_index('ref_hash')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer', rsuffix = '_installs')
aux_ventana1 = pd.read_csv('../xgb/features_auctions_gonzalo_ventana1.csv').set_index('ref_hash')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer', rsuffix = '_auctions')
features_ventana1 = features_ventana1.fillna(0)

labels_sc_ventana1 = pd.read_csv('../xgb/labels_sc_ventana1.csv').set_index('ref_hash')
ventana1 = features_ventana1.join(labels_sc_ventana1, how = 'left').fillna(259200)#3 dias pasado a segundos

In [3]:
filas = ventana1.shape[0]
filas_que_me_quedo = int(filas * 0.4)
matriz = ventana1.iloc[:filas]
ventana1 = 0
features_ventana1 = 0
aux_ventana1 = 0
labels_sc_ventana1 = 0
matriz.shape

(426756, 891)

In [4]:
matriz['attributed_min'] = matriz['attributed_min']*1
matriz['attributed_max'] = matriz['attributed_max']*1
matriz['implicit_min'] = matriz['implicit_min']*1
matriz['implicit_max'] = matriz['implicit_max']*1
matriz['attributed_min'] = matriz['attributed_min'].astype('int64')
matriz['attributed_max'] = matriz['attributed_max'].astype('int64')
matriz['implicit_min'] = matriz['implicit_min'].astype('int64')
matriz['implicit_max'] = matriz['implicit_max'].astype('int64')
matriz['attributed_min'].value_counts()

0    425932
1       824
Name: attributed_min, dtype: int64

In [5]:
features = matriz.iloc[:,:-1]
labels = matriz.iloc[:,-1]

In [6]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = \
    train_test_split(features, labels, test_size=0.25, random_state=123)

In [7]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.055,
                max_depth = 15, alpha = 10, num_boost_round = 50, gamma = 10,
                          n_jobs = 10, n_estimators = 89, nfolds =10 )

In [8]:
xg_reg.fit(train_features,train_labels)

/usr/local/lib/python2.7/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=10, importance_type='gain',
       learning_rate=0.055, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=89, n_jobs=10,
       nfolds=10, nthread=None, num_boost_round=50, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [9]:
predicted_labels = xg_reg.predict(test_features)

In [10]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(test_labels, predicted_labels))
print("RMSE: %f" % (rmse))

RMSE: 52493.413878


In [11]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(50, 60))
xgb.plot_importance(xg_reg, ax=ax)
plt.show()

<Figure size 5000x6000 with 1 Axes>

In [12]:
importances = xg_reg.feature_importances_

In [13]:
importances = xg_reg.feature_importances_
thresholds = importances.tolist()
thresholds.sort()
thresh = thresholds[-50]

In [14]:
columns = features.columns
drop_columns = []
columns[889]
for i in range(0, 890, 1):
    if importances[i] < thresh:
        drop_columns.append(columns[i])
#.drop(drop_columns, axis=1, inplace = True)
drop_columns

['latitude_sum',
 'latitude_mean',
 'latitude_std',
 'latitude_min',
 'latitude_max',
 'longitude_sum',
 'longitude_mean',
 'longitude_std',
 'longitude_min',
 'longitude_max',
 'carrier_id_sum',
 'carrier_id_mean',
 'carrier_id_std',
 'carrier_id_min',
 'carrier_id_max',
 'os_minor_sum',
 'os_minor_mean',
 'os_minor_std',
 'os_minor_min',
 'os_minor_max',
 'agent_device_sum',
 'agent_device_mean',
 'agent_device_std',
 'agent_device_min',
 'agent_device_max',
 'os_major_sum',
 'os_major_mean',
 'os_major_std',
 'os_major_min',
 'os_major_max',
 'specs_brand_sum',
 'specs_brand_mean',
 'specs_brand_std',
 'specs_brand_min',
 'specs_brand_max',
 'timeToClick_sum',
 'timeToClick_mean',
 'timeToClick_std',
 'timeToClick_min',
 'timeToClick_max',
 'hour_sum_clicks',
 'hour_mean_clicks',
 'hour_std_clicks',
 'hour_min_clicks',
 'hour_max_clicks',
 'repeticiones_sum_clicks',
 'repeticiones_mean_clicks',
 'repeticiones_std_clicks',
 'repeticiones_min_clicks',
 'repeticiones_max_clicks',
 'adv